# Protocol definition

In [1]:
# %load STMprotocol
import serial
import struct
import datetime
import time


In [27]:


class STMprotocol:
    def __init__(self, serial_port):
        self.ser = serial.Serial(serial_port, 64000, timeout=0.2)
        self.pack_format = {
            0x01: "=BBBB",
            0x03: "=Bf",
            0x04: "=B",
            0x05: "=B",
            0x06: "=Bffff",
            0x07: "=B",
            0x08: "=fff",
            0x09: "=",
            0x0a: "=",
            0x0b: "=BH",
            
        }

        self.unpack_format = {
            0x01: "=BBBB",
            0x03: "=BB",
            0x04: "=BB",
            0x05: "=BB",
            0x06: "=BB",
            0x07: "=ffff",
            0x08: "=BB",
            0x09: "=fff",
            0x0a: "=fff",
            0x0b: "=BB",
            0x0c: "=f",
                    }

    def pure_send_command(self, cmd, args):
        # Clear buffer
        self.ser.reset_output_buffer()
        self.ser.reset_input_buffer()
        # Sending command
        parameters = bytearray(struct.pack(self.pack_format[cmd], *args))
        msg_len = len(parameters) + 5
        msg = bytearray([0xfa, 0xaf, msg_len, cmd]) + parameters
        crc = sum(msg) % 256
        msg += bytearray([crc])
        self.ser.write(msg)
        if cmd == 176 or cmd == 162:
            print (cmd, args)
        #Receiving data
        data = self.ser.read()
        if len(data) == 0:
            raise Exception("No data received")

        sync = ord(data)
        if sync != 0xfa:
            raise Exception("Incorrect byte of syncronization", sync)
        
        data = self.ser.read()
        if len(data) == 0:
            raise Exception("No adress received")
        adr = ord(data[0])
        
        if adr != 0xfa:
            raise Exception("Incorrect adress", adr)
        answer_len = ord(self.ser.read()[0])
        answer = bytearray(self.ser.read(answer_len - 3))
        
        if (sync + adr + answer_len + sum(answer[:-1])) % 256 != answer[-1]:
            raise Exception("Error with check sum", sync, adr, answer_len, answer)
        args = struct.unpack(self.unpack_format[cmd], answer[1:-1])
        return True, args


    def send_command(self, cmd, args, n_repeats = 5):
        print (cmd, args)
        for i in range(n_repeats):
            try:
                return self.pure_send_command(cmd, args)
            except Exception as exc:
                if i == n_repeats-1:
                    print('Exception:\t', exc)
                    print('At time:\t', datetime.datetime.now())
                    print('cmd:', cmd, 'args:', args)
                    print('--------------------------')
        return False, None
    
    def set_speed(self,v): 
        high=100
        if (v<0): # sign = "+" => forward & sign = "-" => backwards
            dir = 100 # change direction
            v=-v
        else:
            dir = 0
        stopped = [4, dir, 0, 0, 0] # v=0 => stopped
        speed_1 = [4, dir, 0, 0, high]# v=1 => speed1 = [4, dir, 0, 0, 100]
        speed_2 = [4, dir, 0, high, high]# v=2 => speed2 = 
        speed_3 = [4, dir,high,high, high]# v=3 => speed3 = 
        
            
         
        if (v==0): 
            # Stop the conveyor
            f=0
            self.send_command(0x06, stopped)
            t=0
        elif (v == 1):
            # set speed 1
            f=5
            self.send_command(0x06, speed_1)
            t=5.30/100 
        elif(v == 2):
            # set speed 2
            f=10
            self.send_command(0x06, speed_2)
            t=2.60/100#
        else:
        # set speed 3 max
            f=10 # TBD
            self.send_command(0x06, speed_3)
            t=2.60/100# TBD
        return(f,t)

# Move the conveyor dx with speed v
    def move_dx(self,dx,v): 
        
        f,t = self.set_speed(v)
        T= dx *t-2  # T - the time of displacement
        self.set_speed(v)
        time.sleep(T) 
        self.set_speed(0)
        print(dx,'  ',T)
        return(dx,T)


# Work with STM

## Start protocol

In [28]:
#skip
protocol_com.ser.close()

In [29]:
protocol_com = STMprotocol("COM13")

In [30]:
#protocol_com.ser.close()

## Test echo

In [34]:
# Echo
protocol_com.send_command(0x01, [ord(c) for c in 'ECHO'])


1 [69, 67, 72, 79]
Exception:	 No data received
At time:	 2018-03-19 16:29:08.635667
cmd: 1 args: [69, 67, 72, 79]
--------------------------


(False, None)

In [11]:
conveyor = protocol_com

## Debug low level commands

# Move 100 cm

In [27]:
# move dx (in cm) with v
conveyor.move_dx(100,2) # Move 100 cm

6 [4, 0, 0, 100, 100]
Exception:	 No data received
At time:	 2018-03-19 04:37:33.311708
cmd: 6 args: [4, 0, 0, 100, 100]
--------------------------
6 [4, 0, 0, 100, 100]
Exception:	 No data received
At time:	 2018-03-19 04:37:34.342174
cmd: 6 args: [4, 0, 0, 100, 100]
--------------------------
6 [4, 0, 0, 0, 0]
Exception:	 No data received
At time:	 2018-03-19 04:37:35.964585
cmd: 6 args: [4, 0, 0, 0, 0]
--------------------------
100    0.6000000000000001


(100, 0.6000000000000001)

# Stop the conveyor

In [ ]:
# Stop the conveyor
conveyor.set_speed(0)

# Change speed

In [ ]:
# Set speed to speed 1
conveyor.set_speed(1)

In [ ]:
# Set speed to speed 1
conveyor.set_speed(0)

In [ ]:
# Set speed to speed 2
conveyor.set_speed(2)

In [ ]:
# Set speed to speed 1
conveyor.set_speed(0)

In [ ]:
# move dx (in cm) with v
#conveyor.move_dx(30,1)

In [ ]:
# Read speed 
#numberOfmotors = 4
#conveyor.send_command(0x07, [numberOfmotors])

## Set and get conveyor speed in its own coordinate system

In [ ]:
# Set robot speed in its own coordinate system
#Vx = 0.0
#Vy = 0.0
#Wz = 0.0
#conveyor.send_command(0x08, [Vx, Vy, Wz])

In [ ]:
# Set robot speed to 0 in its own coordinate system
#Vx = 0.0
#Vy = 0.0
#Wz = 0.0

#conveyor.send_command(0x08, [Vx, Vy, Wz])

In [ ]:
# Read robot speed in its own coordinate system
#conveyor.send_command(0x09, [])

## Read Robot's coordinates' increments (robot coordinate system)

In [ ]:
# Read robot coord in its own coordinate system
#protocol.send_command(0x0a, [])